In [1]:
import pandas as pd
import numpy as np
import functions
from functions import UniprotDataFetcher
from functions import PDB_analyzer
import requests
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [3]:
def check(uniprotID):
    df = UniprotDataFetcher(uniprotID).convert_structure_to_dataframe()
    # Make sure that there are PDB fiels for a given ID
    if np.isnan(df["PDB_Sequence_Length"]).all():
        return np.nan
    else:
        full_length = UniprotDataFetcher(uniprotID).get_sequence_length()
        full_length_list = [full_length for _ in range(len(df))]
        df["UniProt_Sequence_Length"] = full_length_list
        df["Coverage"], df["Depth"], df["OnlyCarbonAlpha"] = zip(*df.apply(lambda row: PDB_analyzer(row['PDB ID'], row['Chain']).get_coverage_and_CA_info(), axis=1))
        df["Percent_Coverage"] = df['Coverage'] / df['UniProt_Sequence_Length'] * 100
        df = df.dropna(subset=['Coverage'])
        df_sorted = df.sort_values(by=['Coverage', 'Depth', 'Resolution'], ascending=[False, False, True])
        df_sorted = df_sorted.reset_index(drop=True)
        if len(df) == 1:
            return df_sorted
        elif len(df) == 0:
            return np.nan
        else:
            #if 'OnlyCarbonAlpha' in df_sorted.columns:
                #print("Column Exist")
            #else:
                #print("COlumn Does not Exist")
            #print(df_sorted)
            df_filtered =  df_sorted[df_sorted['OnlyCarbonAlpha']==False]
            return df_filtered

def process_dataframe(df):
    # Store the top 4 rows initially
    top_4_rows = df.head(4).copy()
    
    while True:
        # Iterate over the dataframe rows starting from the second row
        for i in range(1, len(df)):
            #print(df)
            # Check if the coverage of the previous row is within +/- 20 of the coverage of the current row
            if abs(df.iloc[i]['Coverage'] - df.iloc[i-1]['Coverage']) <= 20:
                # Check if the depth of the previous row is lower than the depth of the current row
                if df.iloc[i-1]['Depth'] < df.iloc[i]['Depth']:
                    # Swap the rows
                    df.iloc[i-1], df.iloc[i] = df.iloc[i], df.iloc[i-1]
            #if isinstance(df.iloc[i-1]['Resolution'], int) and isinstance(df.iloc[i]['Resolution'], int):
                #print(f"above: {df.iloc[i-1]['Resolution']}")
                #print(f"below: {df.iloc[i]['Resolution']}")
                #print(df.iloc[i-1]["Resolution"])
            if df.iloc[i-1]["Resolution"] != '-' and df.iloc[i]["Resolution"] != '-':
                #print("yes")
                if df.iloc[i-1]['Resolution'] > 4 and df.iloc[i]['Resolution'] < 4:
                    #print("IM FUCKED")
                    df.iloc[i-1], df.iloc[i] = df.iloc[i], df.iloc[i-1]
                        
            #if df.iloc[i-1]['Resolution'] > 4 and df.iloc[i]['Resolution'] < 4:
                #print("hi")
            # Swap the rows
                #df.iloc[i-1], df.iloc[i] = df.iloc[i], df.iloc[i-1]
        
        # Get the new top 4 rows
        new_top_4_rows = df.head(4).copy()
        
        # Check if the top 4 rows haven't changed meaning that the 
        # top 4 rows are the top 4 rows :) 
        if new_top_4_rows.equals(top_4_rows):
            break  # Exit the loop
        
        top_4_rows = new_top_4_rows.copy()  # Update the top 4 rows
    
    return df
# basically like the check function but returns the list of pdb files in order
# 
def get_list_PDB_Ordered(UniprotID):
    filter_1_df = check(UniprotID)
    try:
        filter_2_df = process_dataframe(filter_1_df)
        return filter_2_df["PDB ID"].to_list()
    except AttributeError as e:
        return np.nan



In [5]:
# Sample dataset for the Paper
df = check("P06213") 
df_fil = process_dataframe(df)

In [6]:
df_fil

,PDB ID,Resolution,Locations,Chain,PDB_Sequence_Length,Method,UniProt_Sequence_Length,Coverage,Depth,OnlyCarbonAlpha,Percent_Coverage
0,6PXV,3.2,A/C=28-1382,A,1355,EM,1382,822,909,False,59.479016
1,7YQ3,3.6,E/F=28-946,E,919,EM,1382,829,906,False,59.985528
2,4ZXB,3.3,E=28-956,E,929,X-ray,1382,799,909,False,57.814761
3,7QID,5.0,"A/C=28-746, B/D=763-956","A,B",913,EM,1382,911,911,False,65.918958
4,6SOF,4.3,"A/C=28-746, B/D=795-956","A,B",881,EM,1382,879,879,False,63.603473
5,7MQR,4.1,E/F=28-955,E,928,EM,1382,822,909,False,59.479016
6,8GUY,4.18,E/F=28-946,E,919,EM,1382,834,906,False,60.347323
7,7YQ6,4.18,E/F=28-946,E,919,EM,1382,798,906,False,57.742402
8,7PG2,6.7,A/B=1-1382,A,1382,EM,1382,785,906,False,56.801737
9,7MQO,3.4,E/F=28-956,E,929,EM,1382,601,717,False,43.487699


# Import the whole data Set and filter out any uniprot IDs without PDB files

In [8]:
df_full = pd.read_excel("uniport_human_with_AF2_PDB.xlsx") 
length = len(df_full)
print(f"The total amount of human proteins with alphafold predictions: {length}")
df_full.dropna(subset=["PDB"], inplace=True) 
print(f"Filtered out {length - len(df_full)} proteins that did not have PDB files")
print(f"Final count = {len(df_full)}")
df_full.head(4)

The total amount of human proteins with alphafold predictions: 20422
Filtered out 12606 proteins that did not have PDB files
Final count = 7816


,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,AlphaFoldDB,PDB,3D
1,Q9NQ94,reviewed,A1CF_HUMAN,APOBEC1 complementation factor (APOBEC1-stimul...,A1CF ACF ASP,Homo sapiens (Human),594,Q9NQ94;,2CPD;,NMR spectroscopy (1)
2,P01023,reviewed,A2MG_HUMAN,Alpha-2-macroglobulin (Alpha-2-M) (C3 and PZP-...,A2M CPAMD5 FWP007,Homo sapiens (Human),1474,P01023;,1BV8;2P9R;6TAV;7O7L;7O7M;7O7N;7O7O;7O7P;7O7Q;7...,Electron microscopy (10); NMR spectroscopy (1)...
3,A8K2U0,reviewed,A2ML1_HUMAN,Alpha-2-macroglobulin-like protein 1 (C3 and P...,A2ML1 CPAMD9,Homo sapiens (Human),1454,A8K2U0;,7Q1Y;7Q5Z;7Q60;7Q61;7Q62;,Electron microscopy (4); X-ray crystallography...
7,Q9NRG9,reviewed,AAAS_HUMAN,Aladin (Adracalin),AAAS ADRACALA GL003,Homo sapiens (Human),546,Q9NRG9;,7R5J;7R5K;,Electron microscopy (2)


 - Get only the columns of the original df that I need
 - Split the dataset to 2,000 mini data frames

In [9]:
selected_columns = ['Entry', 'Reviewed', 'Entry Name', 'Protein names', 'Gene Names', 'Organism', 'Length']
# Split the dfs in to 100 mini dfs
df_full_sel = df_full[selected_columns] 
mini_dfs = np.array_split(df_full_sel, 2000)

In [15]:
# only ran from dataset 0 to 194
df_comp = mini_dfs[0]
df_comp['PDB_Files_Ordered'] = df_comp['Entry'].apply(get_list_PDB_Ordered)
print(len(mini_dfs))
for i in range(1, len(mini_dfs)):
    mini_dfs[i]['PDB_Files_Ordered'] = mini_dfs[i]['Entry'].apply(get_list_PDB_Ordered)
    print(f"Finished with {i}th dataset")
    frames = [df_comp, mini_dfs[i]]
    df_comp = pd.concat(frames)
    
    df_comp.to_excel("Compiled_Data_all_supercomputer.xlsx", index=False) 

2000
Finished with 1th dataset
Finished with 2th dataset
Finished with 3th dataset
Finished with 4th dataset
Finished with 5th dataset
Finished with 6th dataset
Finished with 7th dataset
Finished with 8th dataset
Finished with 9th dataset
Finished with 10th dataset
Finished with 11th dataset
Finished with 12th dataset
Finished with 13th dataset
Finished with 14th dataset
Finished with 15th dataset
Finished with 16th dataset
Finished with 17th dataset
Finished with 18th dataset
Finished with 19th dataset
Finished with 20th dataset
Finished with 21th dataset
Finished with 22th dataset
Finished with 23th dataset
Finished with 24th dataset
Finished with 25th dataset
Finished with 26th dataset
Finished with 27th dataset
Finished with 28th dataset
Finished with 29th dataset
Finished with 30th dataset
Finished with 31th dataset
Finished with 32th dataset
Finished with 33th dataset
Finished with 34th dataset
Finished with 35th dataset
Finished with 36th dataset
Finished with 37th dataset
Finis

KeyboardInterrupt: 

In [6]:
# only ran from dataset 195 to 522
df_comp_2 = mini_dfs[195]
df_comp_2['PDB_Files_Ordered'] = df_comp_2['Entry'].apply(get_list_PDB_Ordered)
print(len(mini_dfs))
for i in range(196, len(mini_dfs)):
    mini_dfs[i]['PDB_Files_Ordered'] = mini_dfs[i]['Entry'].apply(get_list_PDB_Ordered)
    print(f"Finished with {i}th dataset")
    frames = [df_comp_2, mini_dfs[i]]
    df_comp_2 = pd.concat(frames)
    
    df_comp_2.to_excel("Compiled_Data_all_supercomputer2.xlsx", index=False) 

2000
Finished with 196th dataset
Finished with 197th dataset
Finished with 198th dataset
Finished with 199th dataset
Finished with 200th dataset
Finished with 201th dataset
Finished with 202th dataset
Finished with 203th dataset
Finished with 204th dataset
Finished with 205th dataset
Finished with 206th dataset
Finished with 207th dataset
Finished with 208th dataset
Finished with 209th dataset
Finished with 210th dataset
Finished with 211th dataset
Finished with 212th dataset
Finished with 213th dataset
Finished with 214th dataset
Finished with 215th dataset
Finished with 216th dataset
Finished with 217th dataset
Finished with 218th dataset
Finished with 219th dataset
Finished with 220th dataset
Finished with 221th dataset
Finished with 222th dataset
Finished with 223th dataset
Finished with 224th dataset
Finished with 225th dataset
Finished with 226th dataset
Finished with 227th dataset
Finished with 228th dataset
Finished with 229th dataset
Finished with 230th dataset
Finished with 2

ValueError: invalid literal for int() with base 10: ''

In [ ]:
# only ran from dataset 522 to 620
df_comp3 = mini_dfs[523]
df_comp3['PDB_Files_Ordered'] = df_comp3['Entry'].apply(get_list_PDB_Ordered)
print(len(mini_dfs))
for i in range(524, len(mini_dfs)):
    mini_dfs[i]['PDB_Files_Ordered'] = mini_dfs[i]['Entry'].apply(get_list_PDB_Ordered)
    print(f"Finished with {i}th dataset")
    frames = [df_comp3, mini_dfs[i]]
    df_comp3 = pd.concat(frames)
    
    df_comp3.to_excel("Compiled_Data_all_supercomputer3.xlsx", index=False) 

2000
Finished with 524th dataset
Finished with 525th dataset
Finished with 526th dataset
Finished with 527th dataset
Finished with 528th dataset
Finished with 529th dataset
Finished with 530th dataset
Finished with 531th dataset
Finished with 532th dataset
Finished with 533th dataset
Finished with 534th dataset
Finished with 535th dataset
Finished with 536th dataset
Finished with 537th dataset
Finished with 538th dataset
Finished with 539th dataset
Finished with 540th dataset
Finished with 541th dataset
Finished with 542th dataset
Finished with 543th dataset
Finished with 544th dataset
Finished with 545th dataset
Finished with 546th dataset
Finished with 547th dataset
Finished with 548th dataset
Finished with 549th dataset
Finished with 550th dataset
Finished with 551th dataset
Finished with 552th dataset
Finished with 553th dataset
Finished with 554th dataset
Finished with 555th dataset
Finished with 556th dataset
Finished with 557th dataset
Finished with 558th dataset
Finished with 5

In [ ]:
# only ran from dataset 621 to 814
df_comp4 = mini_dfs[621]
df_comp4['PDB_Files_Ordered'] = df_comp4['Entry'].apply(get_list_PDB_Ordered)
print(len(mini_dfs))
for i in range(622, len(mini_dfs)):
    mini_dfs[i]['PDB_Files_Ordered'] = mini_dfs[i]['Entry'].apply(get_list_PDB_Ordered)
    print(f"Finished with {i}th dataset")
    frames = [df_comp4, mini_dfs[i]]
    df_comp4 = pd.concat(frames)
    
    df_comp4.to_excel("Compiled_Data_all_supercomputer4.xlsx", index=False) 

2000
Finished with 622th dataset
Finished with 623th dataset
Finished with 624th dataset
Finished with 625th dataset
Finished with 626th dataset
Finished with 627th dataset
Finished with 628th dataset
Finished with 629th dataset
Finished with 630th dataset
Finished with 631th dataset
Finished with 632th dataset
Finished with 633th dataset
Finished with 634th dataset
Finished with 635th dataset
Finished with 636th dataset
Finished with 637th dataset
Finished with 638th dataset
Finished with 639th dataset
Finished with 640th dataset
Finished with 641th dataset
Finished with 642th dataset
Finished with 643th dataset
Finished with 644th dataset
Finished with 645th dataset
Finished with 646th dataset
Finished with 647th dataset
Finished with 648th dataset
Finished with 649th dataset
Finished with 650th dataset
Finished with 651th dataset
Finished with 652th dataset
Finished with 653th dataset
Finished with 654th dataset
Finished with 655th dataset
Finished with 656th dataset
Finished with 6

In [ ]:
# only ran from dataset 815 to 
df_comp5 = mini_dfs[816]
df_comp5['PDB_Files_Ordered'] = df_comp5['Entry'].apply(get_list_PDB_Ordered)
print(len(mini_dfs))
for i in range(817, len(mini_dfs)):
    mini_dfs[i]['PDB_Files_Ordered'] = mini_dfs[i]['Entry'].apply(get_list_PDB_Ordered)
    print(f"Finished with {i}th dataset")
    frames = [df_comp5, mini_dfs[i]]
    df_comp5 = pd.concat(frames)
    
    df_comp5.to_excel("Compiled_Data_all_supercomputer5.xlsx", index=False) 

2000
Finished with 817th dataset
Finished with 818th dataset
Finished with 819th dataset
Finished with 820th dataset
Finished with 821th dataset
Finished with 822th dataset
Finished with 823th dataset
Finished with 824th dataset
Finished with 825th dataset
Finished with 826th dataset
Finished with 827th dataset
Finished with 828th dataset
Finished with 829th dataset
Finished with 830th dataset
Finished with 831th dataset
Finished with 832th dataset
Finished with 833th dataset
Finished with 834th dataset
Finished with 835th dataset
Finished with 836th dataset
Finished with 837th dataset
Finished with 838th dataset
Finished with 839th dataset
Finished with 840th dataset
Finished with 841th dataset
Finished with 842th dataset
Finished with 843th dataset
Finished with 844th dataset
Finished with 845th dataset
Finished with 846th dataset
Finished with 847th dataset
Finished with 848th dataset
Finished with 849th dataset
Finished with 850th dataset
Finished with 851th dataset
Finished with 8

In [63]:
df_1 = pd.read_excel("Compiled_Data_all_supercomputer.xlsx")
df_2 = pd.read_excel("Compiled_Data_all_supercomputer2.xlsx")
df_3 = pd.read_excel("Compiled_Data_all_supercomputer3.xlsx")
df_4 = pd.read_excel("Compiled_Data_all_supercomputer4.xlsx")
df_5 = pd.read_excel("Compiled_Data_all_supercomputer5.xlsx")

In [92]:
combined_df = pd.concat([df_1, df_2, df_3, df_4, df_5], ignore_index=True)
combined_df.drop_duplicates(inplace=True)
combined_df_final = pd.concat([combined_df, missing_entries_df])

In [95]:
combined_df_final.to_excel("FullDataSetPDB_Filed_Ordered.xlsx", index=False) 


In [17]:
combined_df_final = pd.read_excel("FullDataSetPDB_Filed_Ordered.xlsx")

In [18]:
# Get all the rows where the PDB files are null
null_cases = combined_df_final[combined_df_final["PDB_Files_Ordered"].isnull()]
null_cases

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,PDB_Files_Ordered
3,Q9NRG9,reviewed,AAAS_HUMAN,Aladin (Adracalin),AAAS ADRACALA GL003,Homo sapiens (Human),546,NaN
29,P61221,reviewed,ABCE1_HUMAN,ATP-binding cassette sub-family E member 1 (2'...,ABCE1 RLI RNASEL1 RNASELI RNS4I OK/SW-cl.40,Homo sapiens (Human),599,NaN
193,Q8WYP5,reviewed,ELYS_HUMAN,Protein ELYS (Embryonic large molecule derived...,AHCTF1 ELYS TMBS62 MSTP108,Homo sapiens (Human),2266,NaN
547,Q9HBG4,reviewed,VPP4_HUMAN,V-type proton ATPase 116 kDa subunit a 4 (V-AT...,ATP6V0A4 ATP6N1B ATP6N2,Homo sapiens (Human),840,NaN
548,Q99437,reviewed,VATO_HUMAN,V-type proton ATPase 21 kDa proteolipid subuni...,ATP6V0B ATP6F,Homo sapiens (Human),205,NaN
...,...,...,...,...,...,...,...,...
7625,O15213,reviewed,WDR46_HUMAN,WD repeat-containing protein 46 (WD repeat-con...,WDR46 BING4 C6orf11 FP221,Homo sapiens (Human),610,NaN
7631,Q8IWA0,reviewed,WDR75_HUMAN,WD repeat-containing protein 75 (U3 small nucl...,WDR75 UTP17,Homo sapiens (Human),830,NaN
7664,P17861,reviewed,XBP1_HUMAN,X-box-binding protein 1 (XBP-1) (Tax-responsiv...,XBP1 TREB5 XBP2,Homo sapiens (Human),261,NaN
7787,O00488,reviewed,ZN593_HUMAN,Zinc finger protein 593 (Zinc finger protein T86),ZNF593 ZT86,Homo sapiens (Human),134,NaN


# RUN THROUGH 2 WITH ADDITIONAL FILTERS

In [2]:
import pandas as pd
df_full = pd.read_excel("uniport_human_with_AF2_PDB.xlsx") 
length = len(df_full)
print(f"The total amount of human proteins with alphafold predictions: {length}")
df_full.dropna(subset=["PDB"], inplace=True) 
print(f"Filtered out {length - len(df_full)} proteins that did not have PDB files")
print(f"Final count = {len(df_full)}")
df_full.head(4)

The total amount of human proteins with alphafold predictions: 20422
Filtered out 12606 proteins that did not have PDB files
Final count = 7816


,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,AlphaFoldDB,PDB,3D
1,Q9NQ94,reviewed,A1CF_HUMAN,APOBEC1 complementation factor (APOBEC1-stimul...,A1CF ACF ASP,Homo sapiens (Human),594,Q9NQ94;,2CPD;,NMR spectroscopy (1)
2,P01023,reviewed,A2MG_HUMAN,Alpha-2-macroglobulin (Alpha-2-M) (C3 and PZP-...,A2M CPAMD5 FWP007,Homo sapiens (Human),1474,P01023;,1BV8;2P9R;6TAV;7O7L;7O7M;7O7N;7O7O;7O7P;7O7Q;7...,Electron microscopy (10); NMR spectroscopy (1)...
3,A8K2U0,reviewed,A2ML1_HUMAN,Alpha-2-macroglobulin-like protein 1 (C3 and P...,A2ML1 CPAMD9,Homo sapiens (Human),1454,A8K2U0;,7Q1Y;7Q5Z;7Q60;7Q61;7Q62;,Electron microscopy (4); X-ray crystallography...
7,Q9NRG9,reviewed,AAAS_HUMAN,Aladin (Adracalin),AAAS ADRACALA GL003,Homo sapiens (Human),546,Q9NRG9;,7R5J;7R5K;,Electron microscopy (2)


In [3]:
df_full[df_full["Entry"] == "P06213"]

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,AlphaFoldDB,PDB,3D
8111,P06213,reviewed,INSR_HUMAN,Insulin receptor (IR) (EC 2.7.10.1) (CD antige...,INSR,Homo sapiens (Human),1382,P06213;,1GAG;1I44;1IR3;1IRK;1P14;1RQQ;2AUH;2B4S;2HR7;2...,Electron microscopy (23); NMR spectroscopy (1)...


In [20]:
df_full = df_full[~df_full['Entry'].isin(null_cases['Entry'])]

In [21]:
df_full

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,AlphaFoldDB,PDB,3D
1,Q9NQ94,reviewed,A1CF_HUMAN,APOBEC1 complementation factor (APOBEC1-stimul...,A1CF ACF ASP,Homo sapiens (Human),594,Q9NQ94;,2CPD;,NMR spectroscopy (1)
2,P01023,reviewed,A2MG_HUMAN,Alpha-2-macroglobulin (Alpha-2-M) (C3 and PZP-...,A2M CPAMD5 FWP007,Homo sapiens (Human),1474,P01023;,1BV8;2P9R;6TAV;7O7L;7O7M;7O7N;7O7O;7O7P;7O7Q;7...,Electron microscopy (10); NMR spectroscopy (1)...
3,A8K2U0,reviewed,A2ML1_HUMAN,Alpha-2-macroglobulin-like protein 1 (C3 and P...,A2ML1 CPAMD9,Homo sapiens (Human),1454,A8K2U0;,7Q1Y;7Q5Z;7Q60;7Q61;7Q62;,Electron microscopy (4); X-ray crystallography...
13,Q8N5Z0,reviewed,AADAT_HUMAN,Kynurenine/alpha-aminoadipate aminotransferase...,AADAT KAT2 KYAT2,Homo sapiens (Human),425,Q8N5Z0;,2QLR;2R2N;2VGZ;2XH1;3DC1;3UE8;4GDY;4GE4;4GE7;4...,X-ray crystallography (17)
15,Q2M2I8,reviewed,AAK1_HUMAN,AP2-associated protein kinase 1 (EC 2.7.11.1) ...,AAK1 KIAA1048,Homo sapiens (Human),961,Q2M2I8;,4WSQ;5L4Q;5TE0;,X-ray crystallography (3)
...,...,...,...,...,...,...,...,...,...,...
20288,O00370,reviewed,LORF2_HUMAN,LINE-1 retrotransposable element ORF2 protein ...,NaN,Homo sapiens (Human),1275,O00370;,1VYB;2V0R;2V0S;7N8K;7N8S;7N94;,X-ray crystallography (6)
20299,P0DOX3,reviewed,IGD_HUMAN,Immunoglobulin delta heavy chain (Immunoglobul...,NaN,Homo sapiens (Human),512,P0DOX3;,1ZVO;,X-ray crystallography (1)
20303,P0DOX5,reviewed,IGG1_HUMAN,Immunoglobulin gamma-1 heavy chain (Immunoglob...,NaN,Homo sapiens (Human),449,P0DOX5;,1N0X;3PGF;4R2G;5O4E;5VJ6;5VU0;5VZX;5VZY;5W5L;5...,Electron microscopy (14); X-ray crystallograph...
20325,P0DOX8,reviewed,IGL1_HUMAN,Immunoglobulin lambda-1 light chain (Immunoglo...,NaN,Homo sapiens (Human),216,P0DOX8;,1MCD;1MCE;,X-ray crystallography (2)


In [22]:
selected_columns = ['Entry', 'Reviewed', 'Entry Name', 'Protein names', 'Gene Names', 'Organism', 'Length']
# Split the dfs in to 100 mini dfs
df_full_sel = df_full[selected_columns] 
mini_dfs = np.array_split(df_full_sel, 2000)

In [41]:
# only ran from dataset 0 to 194
df_comp = mini_dfs[0]
df_comp['PDB_Files_Ordered'] = df_comp['Entry'].apply(get_list_PDB_Ordered)
print(len(mini_dfs))
for i in range(1, len(mini_dfs)):
    mini_dfs[i]['PDB_Files_Ordered'] = mini_dfs[i]['Entry'].apply(get_list_PDB_Ordered)
    print(f"Finished with {i}th dataset")
    frames = [df_comp, mini_dfs[i]]
    df_comp = pd.concat(frames)
    
    df_comp.to_excel("secondrun1.xlsx", index=False) 

2000
Finished with 1th dataset
Finished with 2th dataset
Finished with 3th dataset
Finished with 4th dataset
Finished with 5th dataset
Finished with 6th dataset
Finished with 7th dataset
Finished with 8th dataset


TypeError: '>' not supported between instances of 'str' and 'int'

In [62]:
# only ran from dataset 9 to 98
df_comp = mini_dfs[9]
df_comp['PDB_Files_Ordered'] = df_comp['Entry'].apply(get_list_PDB_Ordered)
print(len(mini_dfs))
for i in range(10, len(mini_dfs)):
    mini_dfs[i]['PDB_Files_Ordered'] = mini_dfs[i]['Entry'].apply(get_list_PDB_Ordered)
    print(f"Finished with {i}th dataset")
    frames = [df_comp, mini_dfs[i]]
    df_comp = pd.concat(frames)
    
    df_comp.to_excel("secondrun2.xlsx", index=False) 

2000
Finished with 10th dataset
Finished with 11th dataset
Finished with 12th dataset
Finished with 13th dataset
Finished with 14th dataset
Finished with 15th dataset
Finished with 16th dataset
Finished with 17th dataset
Finished with 18th dataset
Finished with 19th dataset
Finished with 20th dataset
Finished with 21th dataset
Finished with 22th dataset
Finished with 23th dataset
Finished with 24th dataset
Finished with 25th dataset
Finished with 26th dataset
Finished with 27th dataset
Finished with 28th dataset
Finished with 29th dataset
Finished with 30th dataset
Finished with 31th dataset
Finished with 32th dataset
Finished with 33th dataset
Finished with 34th dataset
Finished with 35th dataset
Finished with 36th dataset
Finished with 37th dataset
Finished with 38th dataset
Finished with 39th dataset
Finished with 40th dataset
Finished with 41th dataset
Finished with 42th dataset
Finished with 43th dataset
Finished with 44th dataset
Finished with 45th dataset
Finished with 46th data

KeyboardInterrupt: 

In [ ]:
# only ran from dataset 9 to 
df_comp = mini_dfs[99]
df_comp['PDB_Files_Ordered'] = df_comp['Entry'].apply(get_list_PDB_Ordered)
print(len(mini_dfs))
for i in range(100, len(mini_dfs)):
    mini_dfs[i]['PDB_Files_Ordered'] = mini_dfs[i]['Entry'].apply(get_list_PDB_Ordered)
    print(f"Finished with {i}th dataset")
    frames = [df_comp, mini_dfs[i]]
    df_comp = pd.concat(frames)
    
    df_comp.to_excel("secondrun3.xlsx", index=False) 

2000
Finished with 100th dataset
Finished with 101th dataset
Finished with 102th dataset
Finished with 103th dataset
Finished with 104th dataset
Finished with 105th dataset
Finished with 106th dataset
Finished with 107th dataset
Finished with 108th dataset
Finished with 109th dataset
Finished with 110th dataset
Finished with 111th dataset
Finished with 112th dataset
Finished with 113th dataset
Finished with 114th dataset
Finished with 115th dataset
Finished with 116th dataset
Finished with 117th dataset
Finished with 118th dataset
Finished with 119th dataset
Finished with 120th dataset
Finished with 121th dataset
Finished with 122th dataset
Finished with 123th dataset
Finished with 124th dataset
Finished with 125th dataset
Finished with 126th dataset
Finished with 127th dataset
Finished with 128th dataset
Finished with 129th dataset
Finished with 130th dataset
Finished with 131th dataset
Finished with 132th dataset
Finished with 133th dataset
Finished with 134th dataset
Finished with 1

In [4]:
pd.read_excel("secondrun3.xlsx")

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,PDB_Files_Ordered
0,P55064,reviewed,AQP5_HUMAN,Aquaporin-5 (AQP-5),AQP5,Homo sapiens (Human),265,"['3D9S', '7STC', '5C5X', '5DYE']"
1,O14520,reviewed,AQP7_HUMAN,Aquaporin-7 (AQP-7) (Aquaglyceroporin-7) (Aqua...,AQP7 AQP7L AQP9,Homo sapiens (Human),342,"['8AMW', '8AMX', '6KXW', '6N1G', '6QZI', '6QZJ']"
2,O60306,reviewed,AQR_HUMAN,RNA helicase aquarius (EC 3.6.4.13) (Intron-bi...,AQR KIAA0560,Homo sapiens (Human),1485,"['6ID1', '6ID0', '6ICZ', '7W59', '7W5A', '4PJ3..."
3,P10275,reviewed,ANDR_HUMAN,Androgen receptor (Dihydrotestosterone recepto...,AR DHTR NR3C4,Homo sapiens (Human),920,"['3V49', '3V4A', '1XOW', '2Q7K', '2Q7I', '5CJ6..."
4,P10398,reviewed,ARAF_HUMAN,Serine/threonine-protein kinase A-Raf (EC 2.7....,ARAF ARAF1 PKS PKS2,Homo sapiens (Human),606,"['1WXM', '2MSE']"
...,...,...,...,...,...,...,...,...
4079,P29475,reviewed,NOS1_HUMAN,"Nitric oxide synthase, brain (EC 1.14.13.39) (...",NOS1,Homo sapiens (Human),1434,"['6CIC', '6CID', '7TS5', '7TS1', '6PNC', '7TS7..."
4080,P35228,reviewed,NOS2_HUMAN,"Nitric oxide synthase, inducible (EC 1.14.13.3...",NOS2 NOS2A,Homo sapiens (Human),1153,"['3E7G', '4NOS', '3EJ8', '1NSI', '4CX7', '2NSI..."
4081,P29474,reviewed,NOS3_HUMAN,"Nitric oxide synthase, endothelial (EC 1.14.13...",NOS3,Homo sapiens (Human),1203,"['6AV7', '6NH5', '6NH3', '6PP2', '6POV', '6POW..."
4082,Q8IVI9,reviewed,NOSTN_HUMAN,Nostrin (BM247 homolog) (Nitric oxide synthase...,NOSTRIN,Homo sapiens (Human),506,['2YUN']


In [ ]:
# only ran from dataset 9 to 
df_comp = mini_dfs[1120]
df_comp['PDB_Files_Ordered'] = df_comp['Entry'].apply(get_list_PDB_Ordered)
print(len(mini_dfs))
for i in range(1121, len(mini_dfs)):
    mini_dfs[i]['PDB_Files_Ordered'] = mini_dfs[i]['Entry'].apply(get_list_PDB_Ordered)
    print(f"Finished with {i}th dataset")
    frames = [df_comp, mini_dfs[i]]
    df_comp = pd.concat(frames)
    
    df_comp.to_excel("secondrun4.xlsx", index=False) 

2000
Finished with 1121th dataset
Finished with 1122th dataset
Finished with 1123th dataset
Finished with 1124th dataset
Finished with 1125th dataset
Finished with 1126th dataset
Finished with 1127th dataset
Finished with 1128th dataset
Finished with 1129th dataset
Finished with 1130th dataset
Finished with 1131th dataset
Finished with 1132th dataset
Finished with 1133th dataset
Finished with 1134th dataset
Finished with 1135th dataset
Finished with 1136th dataset
Finished with 1137th dataset
Finished with 1138th dataset
Finished with 1139th dataset
Finished with 1140th dataset
Finished with 1141th dataset
Finished with 1142th dataset
Finished with 1143th dataset
Finished with 1144th dataset
Finished with 1145th dataset
Finished with 1146th dataset
Finished with 1147th dataset
Finished with 1148th dataset
Finished with 1149th dataset
Finished with 1150th dataset
Finished with 1151th dataset
Finished with 1152th dataset
Finished with 1153th dataset
Finished with 1154th dataset
Finished 

In [12]:
# find where the code left off

target_value = "P57060"

for i in range(1000, len(mini_dfs)):
    if target_value in mini_dfs[i]["Entry"].values:
        print(i)

1477


In [10]:
mini_dfs[1327]

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length
13722,P07602,reviewed,SAP_HUMAN,Prosaposin (Proactivator polypeptide) [Cleaved...,PSAP GLBA SAP1,Homo sapiens (Human),524
13724,Q9Y617,reviewed,SERC_HUMAN,Phosphoserine aminotransferase (EC 2.6.1.52) (...,PSAT1 PSA,Homo sapiens (Human),370
13730,P49768,reviewed,PSN1_HUMAN,Presenilin-1 (PS-1) (EC 3.4.23.-) (Protein S18...,PSEN1 AD3 PS1 PSNL1,Homo sapiens (Human),467
13731,P49810,reviewed,PSN2_HUMAN,Presenilin-2 (PS-2) (EC 3.4.23.-) (AD3LP) (AD5...,PSEN2 AD4 PS2 PSNL2 STM2,Homo sapiens (Human),448


In [11]:
pd.read_excel("secondrun5.xlsx")
df

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,PDB_Files_Ordered
0,O60894,reviewed,RAMP1_HUMAN,Receptor activity-modifying protein 1 (Calcito...,RAMP1,Homo sapiens (Human),148,"['6ZHO', '7P0I', '8AX7', '6ZIS', '8AX6', '4RWG..."
1,O60895,reviewed,RAMP2_HUMAN,Receptor activity-modifying protein 2 (Calcito...,RAMP2,Homo sapiens (Human),175,"['4RWF', '6V2E', '7TYX', '6UUN', '7TYH', '7TYY..."
2,O60896,reviewed,RAMP3_HUMAN,Receptor activity-modifying protein 3 (Calcito...,RAMP3,Homo sapiens (Human),148,"['7TZF', '6UVA', '6UUS']"
3,P62826,reviewed,RAN_HUMAN,GTP-binding nuclear protein Ran (EC 3.6.5.-) (...,RAN ARA24 OK/SW-cl.81,Homo sapiens (Human),216,"['6A3A', '6A3C', '7MNW', '6A3B', '6KFT', '6A38..."
4,P43487,reviewed,RANG_HUMAN,Ran-specific GTPase-activating protein (Ran-bi...,RANBP1,Homo sapiens (Human),201,"['1K5D', '1K5G']"
...,...,...,...,...,...,...,...,...
363,Q9Y265,reviewed,RUVB1_HUMAN,RuvB-like 1 (EC 3.6.4.12) (49 kDa TATA box-bin...,RUVBL1 INO80H NMP238 TIP49 TIP49A,Homo sapiens (Human),456,"['6K0R', '2XSZ', '7OLE', '2C9O', '6HTS', '5OAF..."
364,Q9Y230,reviewed,RUVB2_HUMAN,RuvB-like 2 (EC 3.6.4.12) (48 kDa TATA box-bin...,RUVBL2 INO80J TIP48 TIP49B CGI-46,Homo sapiens (Human),463,"['6K0R', '3UK6', '2XSZ', '7OLE', '6HTS', '5OAF..."
365,Q9H446,reviewed,RWDD1_HUMAN,RWD domain-containing protein 1 (DRG family-re...,RWDD1 DFRP2 CGI-24 PTD013,Homo sapiens (Human),243,['2EBM']
366,Q9UIY3,reviewed,RWD2A_HUMAN,RWD domain-containing protein 2A,RWDD2A RWDD2,Homo sapiens (Human),292,['2DAW']


In [ ]:
# only ran from dataset 9 to 
df_comp = mini_dfs[1478]
df_comp['PDB_Files_Ordered'] = df_comp['Entry'].apply(get_list_PDB_Ordered)
print(len(mini_dfs))
for i in range(1387, len(mini_dfs)):
    mini_dfs[i]['PDB_Files_Ordered'] = mini_dfs[i]['Entry'].apply(get_list_PDB_Ordered)
    print(f"Finished with {i}th dataset")
    frames = [df_comp, mini_dfs[i]]
    df_comp = pd.concat(frames)
    
    df_comp.to_excel("secondrun5.xlsx", index=False) 

2000
Finished with 1387th dataset
Finished with 1388th dataset
Finished with 1389th dataset
Finished with 1390th dataset
Finished with 1391th dataset
Finished with 1392th dataset
Finished with 1393th dataset
Finished with 1394th dataset
Finished with 1395th dataset
Finished with 1396th dataset
Finished with 1397th dataset
Finished with 1398th dataset
Finished with 1399th dataset
Finished with 1400th dataset
Finished with 1401th dataset
Finished with 1402th dataset
Finished with 1403th dataset
Finished with 1404th dataset
Finished with 1405th dataset
Finished with 1406th dataset
Finished with 1407th dataset
Finished with 1408th dataset
Finished with 1409th dataset
Finished with 1410th dataset
Finished with 1411th dataset
Finished with 1412th dataset
Finished with 1413th dataset
Finished with 1414th dataset
Finished with 1415th dataset
Finished with 1416th dataset
Finished with 1417th dataset
Finished with 1418th dataset
Finished with 1419th dataset
Finished with 1420th dataset
Finished 

In [ ]:
# only ran from dataset 9 to 
df_comp = mini_dfs[1478]
df_comp['PDB_Files_Ordered'] = df_comp['Entry'].apply(get_list_PDB_Ordered)
print(len(mini_dfs))
for i in range(1479, len(mini_dfs)):
    mini_dfs[i]['PDB_Files_Ordered'] = mini_dfs[i]['Entry'].apply(get_list_PDB_Ordered)
    print(f"Finished with {i}th dataset")
    frames = [df_comp, mini_dfs[i]]
    df_comp = pd.concat(frames)
    
    df_comp.to_excel("secondrun6.xlsx", index=False) 

2000
Finished with 1479th dataset
Finished with 1480th dataset
Finished with 1481th dataset
Finished with 1482th dataset
Finished with 1483th dataset
Finished with 1484th dataset
Finished with 1485th dataset
Finished with 1486th dataset
Finished with 1487th dataset
Finished with 1488th dataset
Finished with 1489th dataset
Finished with 1490th dataset
Finished with 1491th dataset
Finished with 1492th dataset
Finished with 1493th dataset
Finished with 1494th dataset
Finished with 1495th dataset
Finished with 1496th dataset
Finished with 1497th dataset
Finished with 1498th dataset
Finished with 1499th dataset
Finished with 1500th dataset
Finished with 1501th dataset
Finished with 1502th dataset
Finished with 1503th dataset
Finished with 1504th dataset
Finished with 1505th dataset
Finished with 1506th dataset
Finished with 1507th dataset
Finished with 1508th dataset
Finished with 1509th dataset
Finished with 1510th dataset
Finished with 1511th dataset
Finished with 1512th dataset
Finished 

In [24]:
end = 1790 
mini_dfs[end]


,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length
18064,P17752,reviewed,TPH1_HUMAN,Tryptophan 5-hydroxylase 1 (EC 1.14.16.4) (Try...,TPH1 TPH TPRH TRPH,Homo sapiens (Human),444
18065,Q8IWU9,reviewed,TPH2_HUMAN,Tryptophan 5-hydroxylase 2 (EC 1.14.16.4) (Neu...,TPH2 NTPH,Homo sapiens (Human),490
18066,P60174,reviewed,TPIS_HUMAN,Triosephosphate isomerase (TIM) (EC 5.3.1.1) (...,TPI1 TPI,Homo sapiens (Human),249


In [25]:
pd.read_excel("secondrun6.xlsx")

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,PDB_Files_Ordered
0,Q9Y3V2,reviewed,RWDD3_HUMAN,RWD domain-containing protein 3 (RWD domain-co...,RWDD3 RSUME,Homo sapiens (Human),267,['4Y1L']
1,Q9HBX9,reviewed,RXFP1_HUMAN,Relaxin receptor 1 (Leucine-rich repeat-contai...,RXFP1 LGR7,Homo sapiens (Human),757,['7TMW']
2,Q8WXD0,reviewed,RXFP2_HUMAN,Relaxin receptor 2 (G-protein coupled receptor...,RXFP2 GPR106 GREAT LGR8,Homo sapiens (Human),754,['2M96']
3,P19793,reviewed,RXRA_HUMAN,Retinoic acid receptor RXR-alpha (Nuclear rece...,RXRA NR2B1,Homo sapiens (Human),462,"['4NQA', '3DZY', '5UAN', '3E00', '3DZU', '4J5X..."
4,P28702,reviewed,RXRB_HUMAN,Retinoic acid receptor RXR-beta (Nuclear recep...,RXRB NR2B2,Homo sapiens (Human),533,"['5HJP', '5KYJ', '5KYA', '5I4V', '7A78', '1UHL..."
...,...,...,...,...,...,...,...,...
1689,O00370,reviewed,LORF2_HUMAN,LINE-1 retrotransposable element ORF2 protein ...,NaN,Homo sapiens (Human),1275,"['1VYB', '2V0R', '7N8K', '7N8S', '7N94', '2V0S']"
1690,P0DOX3,reviewed,IGD_HUMAN,Immunoglobulin delta heavy chain (Immunoglobul...,NaN,Homo sapiens (Human),512,['1ZVO']
1691,P0DOX5,reviewed,IGG1_HUMAN,Immunoglobulin gamma-1 heavy chain (Immunoglob...,NaN,Homo sapiens (Human),449,"['8DV1', '8DV2', '7CZV', '7CZU', '5O4E', '6BFT..."
1692,P0DOX8,reviewed,IGL1_HUMAN,Immunoglobulin lambda-1 light chain (Immunoglo...,NaN,Homo sapiens (Human),216,"['1MCE', '1MCD']"


In [26]:
# find where the code left off

target_value = "P0DOX7"

for i in range(1000, len(mini_dfs)):
    if target_value in mini_dfs[i]["Entry"].values:
        print(i)

1999
